# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [3]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from CheXpert.shared_utils import vprint, to_gpu
from CheXpert import shared_utils
from CheXpert.disease_prediction.utils import Configs as disease_configs
from CheXpert.race_prediction.utils import Configs as race_configs

# Configs

In [4]:
@dataclass
class Configs:
    DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    VALID_LABELS_FILENAME = "valid.csv"
    DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_CLASSES = disease_configs.ANNOTATIONS_COLUMNS
    RACE_CLASSES = race_configs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = disease_configs.NUM_CLASSES
    NUM_RACE_CLASSES = race_configs.NUM_CLASSES
    BATCH_SIZE = 4
    SEED = 123
    VERBOSE = 1
    VALID_SIZE_DEBUG = -1

In [5]:
shared_utils.set_seed(Configs.SEED)

In [6]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-22 11:22: Memory info: 8.5 GB free GPU.


In [7]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

In [8]:
disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.DATA_DIR, 
                                               labels_filename=Configs.VALID_LABELS_FILENAME,
                                               transform=valid_transform)
disease_valid_dataset.df_labels = disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG] # hack for speed debugging
disease_valid_dataloader = DataLoader(disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(disease_valid_dataset)

233

In [9]:
_, _, files = next(os.walk(Configs.DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [11]:
disease_model = densenet121()
num_features = disease_model.classifier.in_features
disease_model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
disease_model.eval()
not disease_model.training

True

In [12]:
disease_model, results, _, _ = shared_utils.load_statedict(disease_model, disease_trained_models[0], Configs)
disease_model = to_gpu(disease_model)

2022-07-22 11:22: Loading model - CheXpert/disease_prediction/trained_models/2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


In [13]:
disease_labels, disease_outputs = shared_utils.get_metric_tensors(disease_model, disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)

In [59]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_CLASSES, 
                              data=[shared_utils.auc_score(disease_labels, disease_outputs, per_class=True),],
                             index=['AUC'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
AUC,0.837498,0.833868,0.932727,0.939007,0.936763


# Race Prediction

In [21]:
# Create data loaders.
race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.DATA_DIR, demo_filename=Configs.DEMO_FILENAME,
                                       labels_filename=Configs.VALID_LABELS_FILENAME,
                                       transform=valid_transform)
race_valid_dataloader = DataLoader(race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
Configs.TRAIN_LOADER_SIZE = len(race_valid_dataloader)
len(race_valid_dataset)

172

In [22]:
_, _, files = next(os.walk(Configs.RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

8

In [23]:
df_res_race = pd.DataFrame(columns=Configs.RACE_CLASSES)
df_res_race

,White,Asian,Black,Hispanic


In [24]:
for model_version, model_path in zip(race_model_versions, race_trained_models):
    model = densenet121()
    
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model = to_gpu(model)
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs)
    labels, outputs = shared_utils.get_metric_tensors(model, race_valid_dataloader, Configs,
                                                      apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                      by_study=False, challenge_ann_only=None)
    auc_scores = shared_utils.auc_score(labels, outputs, per_class=True)
    df_res_race.loc[model_version] = auc_scores

df_res_race   

2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-15_29__densenet121_race_denseblock1_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2805__validAUC-0.9178.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811__validAUC-0.9094.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_21-13_21__densenet121_race_denseblock2_shallow__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3791__validAUC-0.9075.dict
2022-07-22 11:35: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__

,White,Asian,Black,Hispanic
densenet121_race_denseblock1_freezed,0.922287,0.895706,0.955357,0.897821
densenet121_race_denseblock2_freezed,0.904110,0.873892,0.973214,0.886447
densenet121_race_classifier_freezed,0.796628,0.583504,0.666667,0.744361
densenet121_race_denseblock2_shallow,0.882508,0.909339,0.944940,0.893195
densenet121_race_denseblock3_freezed,0.898841,0.805726,0.959821,0.876807
densenet121_race_denseblock4_freezed,0.893836,0.775733,0.937500,0.841334
densenet121_race_denseblock1_shallow,0.919916,0.799591,0.888393,0.868517
densenet121_race,0.904900,0.934560,0.958333,0.880663


In [28]:
df_res_race["mean"] = df_res_race.mean(axis=1)
df_res_race.sort_values(by="mean", ascending=False, inplace=True)
df_res_race = df_res_race.round(2)

In [29]:
df_res_race

,White,Asian,Black,Hispanic,mean
densenet121_race,0.90,0.93,0.96,0.88,0.92
densenet121_race_denseblock1_freezed,0.92,0.90,0.96,0.90,0.92
densenet121_race_denseblock2_freezed,0.90,0.87,0.97,0.89,0.91
densenet121_race_denseblock2_shallow,0.88,0.91,0.94,0.89,0.91
densenet121_race_denseblock3_freezed,0.90,0.81,0.96,0.88,0.89
densenet121_race_denseblock1_shallow,0.92,0.80,0.89,0.87,0.87
densenet121_race_denseblock4_freezed,0.89,0.78,0.94,0.84,0.86
densenet121_race_classifier_freezed,0.80,0.58,0.67,0.74,0.70


# Disease Per Race Performance

In [55]:
df_race = race_valid_dataset.df_labels[race_valid_dataset.df_labels.columns[:10]].drop_duplicates()
df_joined = disease_valid_dataset.df_labels.merge(df_race, how='left', left_on='patient_id', right_on='PATIENT')
df_joined.race.fillna('Other', inplace=True) 
df_joined.head(2)

,original_path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_path,patient_id,study,view,PATIENT,GENDER,AGE_AT_CXR,PRIMARY_RACE,ETHNICITY,race,Asian,Black,Hispanic,White
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64541,study1,view1_frontal.jpg,patient64541,Male,73.0,White,Non-Hispanic/Non-Latino,White,0.0,0.0,0.0,1.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64542,study1,view1_frontal.jpg,patient64542,Male,70.0,White,Non-Hispanic/Non-Latino,White,0.0,0.0,0.0,1.0


In [61]:
df_joined.race.value_counts()

White       132
Other        62
Asian        26
Black         9
Hispanic      4
Name: race, dtype: int64

In [60]:
for race in df_joined.race.unique():
    disease_race_labels = disease_labels[df_joined.race==race]
    disease_race_outputs = disease_outputs[df_joined.race==race]
    df_res_disease.loc[race] = shared_utils.auc_score(disease_race_labels, disease_race_outputs, per_class=True)
df_res_disease

Only one class present in y_true. ROC AUC score is not defined in that case.


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
AUC,0.837498,0.833868,0.932727,0.939007,0.936763
White,0.852639,0.820924,0.935650,0.941308,0.922452
Asian,0.803922,0.849673,0.886364,1.000000,0.921569
Other,0.776786,0.836310,0.928571,0.909615,0.983007
Hispanic,1.000000,0.750000,1.000000,-1.000000,1.000000
Black,1.000000,1.000000,1.000000,0.900000,1.000000


In [16]:
df_disease = pd.DataFrame(columns=Configs.DISEASE_CLASSES, 
                              data=[shared_utils.auc_score(disease_labels, disease_outputs, per_class=True),],
                              index=['all_races'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
all_races,0.837498,0.833868,0.932727,0.939007,0.936763
